In [1]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC  
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

import os,json
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Flatten, Input, Conv1D,MaxPooling1D,Embedding
from keras.models import load_model, Model
from keras.utils import to_categorical
from sklearn.metrics import classification_report,confusion_matrix
import pandas as pd
import sqlalchemy as sa
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import gensim
from nltk.corpus import stopwords
import collections
from collections import defaultdict
from sklearn.preprocessing import MultiLabelBinarizer
import nltk
user = 'dev_ds'
pwd = 'akBefvuIj4'
from base64 import b64decode
conn_str = 'mssql+pyodbc://%s:%s@repdb1irv.iherb.net/iHerb_Shop?driver=ODBC+Driver+13+for+SQL+Server'
engine = sa.create_engine(conn_str % ('dev_ds', b64decode('YWtCZWZ2dUlqNA==').decode()))

import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

C:\Users\fjavadi\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\fjavadi\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#Removing the main first level categories
#ParentCategoryID=8120 : Brands, HealthTopics=
p='inventoryvelocity'
query3='''
SELECT A.ProductID as ProductID, A.ParentCategoryID as CategoryID, B.level as level, B.d as d, B.Name as Name 
FROM `inventoryvelocity.input.product_parent_categories_relation` AS A
Inner join `rec-serv.AutoSuggestion.category_data` AS B on A.ParentCategoryID=B.CategoryID 
Where B.VisibleToCustomer=True and B.CategoryID not in (1475,1855,2089,2203,2236,2320,2992,3213,100477,100483,2430,2493,2465,8120)
and B.ParentCategoryID not in (8120,2430)
'''
products_OldCategories=pd.read_gbq(query3,p,dialect='standard')

product_attr = pd.read_sql(''' 
SELECT [ProductID]
      ,[DisplayName]
      ,[Ingredients]
      ,[Description]
      ,[SupplementFacts]
      ,[ServingSize]
      ,[PackageQuantity]
      ,[SuggestedUse]
      ,[Keywords]
  FROM [iHerb_Shop].[shopservice].[TBL_Products]
  where Status in (0,3,4,6)
''', engine)
#product_attr.set_index('ProductID' , inplace=True)
print(len(product_attr))
products_OldCategories=products_OldCategories[products_OldCategories['ProductID'].isin(product_attr.index.values)]
print(products_OldCategories['ProductID'].nunique())

30070
11437


In [3]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', ' ', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isnumeric() or isinstance(word, float):
            new_word=' '
            new_words.append(new_word)
        else:
            word1=word
            new_words.append(word1)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word)
        if(lemma!=" "):
            lemmas.append(lemma)
    return lemmas

def normalize(s):
    s = re.sub(r'[^\w\s]',' ',s)
    words = nltk.word_tokenize(s)  
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words= lemmatize_verbs(words)
    return " ".join(words)

In [4]:
query3='''
SELECT A.ProductID as ProductID, A.ParentCategoryID as CategoryID, B.level as level, B.d as d, B.Name as Name 
FROM `inventoryvelocity.input.product_parent_categories_relation` AS A
Inner join `rec-serv.AutoSuggestion.category_data` AS B on A.ParentCategoryID=B.CategoryID
and B.ParentCategoryID in (1855) and CategoryID not in (101129,100865,101072,8736)
'''
products_supplements=pd.read_gbq(query3,p,dialect='standard')

products_supplements=products_supplements[products_supplements['ProductID'].isin(product_attr.index.values)]
list_supplements_products=products_supplements['ProductID'].unique().tolist()

result_categories = pd.merge( product_attr, products_supplements, on='ProductID', how='inner')
#product_attr.set_index('ProductID' , inplace=True)
result_categories['DisplayName']=result_categories['DisplayName'].apply(lambda x: normalize(x))
result_categories =result_categories[~ result_categories.isnull()]
result_categories = result_categories[~ result_categories['DisplayName'].isnull()]
result_categories=result_categories.groupby(['ProductID','DisplayName'],as_index=False)['CategoryID'].apply(list).reset_index()\
.rename(columns={0:'CategoryID'})
print(result_categories.columns)

Index(['ProductID', 'DisplayName', 'CategoryID'], dtype='object')


In [5]:
Category_results = result_categories['CategoryID'].values.tolist() 
mlb = MultiLabelBinarizer()
binary_categories=mlb.fit_transform(Category_results)
print(mlb.classes_)


[  1476   1542   1694   1800   1930   3282   8738   8741 100727 100804
 100821 100858 100861 100945 101005]


In [6]:
#CategoryID_matrix = result_categories.as_matrix(columns=[result_categories[3:]])

#print(result_categories.iloc[:,[2]].as_matrix())

#categoryId_matrix = result_categories.iloc[:,[2]].as_matrix()
#X_train, X_test, y_train, y_test =train_test_split( result_categories['DisplayName'], result_categories['CategoryID'], test_size=0.33, random_state=42)
X_train, X_test, y_train, y_test =train_test_split( result_categories['DisplayName'], binary_categories , test_size=0.33, random_state=42)
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(X_train)
vectorizer.fit(X_test)
x_train = vectorizer.transform(X_train)
x_test= vectorizer.transform(X_test)

In [7]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB(),require_dense = [True, True]
)
# train
classifier.fit(x_train, y_train)
# predict
predictions = classifier.predict(x_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))


Accuracy =  0.7083764219234746


In [9]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier.fit(x_train, y_train)
# predict
predictions = classifier.predict(x_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

Accuracy =  0.3360910031023785




In [11]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(LogisticRegression())
# train
classifier.fit(x_train, y_train)
# predict
predictions = classifier.predict(x_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
# using Label Powerset
print(classification_report(y_test,predictions))


print("\n")


Accuracy =  0.7218200620475698
             precision    recall  f1-score   support

          0       0.87      0.70      0.77       138
          1       0.89      0.95      0.92       106
          2       0.78      0.82      0.80        71
          3       0.54      0.98      0.70       163
          4       1.00      0.48      0.65        21
          5       1.00      0.48      0.65        48
          6       1.00      0.45      0.62        33
          7       0.77      0.63      0.69        98
          8       0.95      0.55      0.69        77
          9       1.00      0.39      0.56        59
         10       1.00      0.48      0.65        25
         11       0.77      0.82      0.79       111
         12       1.00      0.39      0.56        36
         13       1.00      0.38      0.55        21
         14       0.98      0.80      0.88        59

avg / total       0.84      0.71      0.73      1066





In [50]:
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix
classifier_new = MLkNN(k=10)
# Note that this classifier can throw up errors when handling sparse matrices.
x_train = lil_matrix(x_train).toarray()
y_train = lil_matrix(y_train).toarray()
x_test = lil_matrix(x_test).toarray()
# train
classifier_new.fit(x_train, y_train)
# predict
predictions_new = classifier_new.predict(x_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions_new))
print("\n")


Accuracy =  0.6026694045174538


